# Convert Local Data to an AI training Set
## Example: From USGS dataset to STEAD(HDF5 + CSV)format
Author: Hao Mai 

E-mail: hmai090@uottawa.ca

Oct 04 2022

Here is USGS dataset for first arriving P,Pg,Pn,S,Sg,Sn from the PDE (2013+):
https://www.sciencebase.gov/catalog/item/6127b30fd34e40dd9c05094c
You can download one month zip file to try following codes, e.g., [2013 June zip](https://www.sciencebase.gov/catalog/item/61fc238ad34e622189cb4240) is the smallest size.

-In general, a public dataset will offer user a csv file which includes all metadata, i.e., catalog / event related informations.
-To deploy a AI-based phase picking projects, in most time, we do not need that much information, we only concern a few of them.
-Most important thing is we need to trim the original seismograms(traces) to the same format in model, e.g., STEAD format is one of the most popular used one.
Okay, Let's start to build it!



In [1]:
import h5py
import csv
import numpy as np
import glob
import pandas as pd
import time
import re
from obspy.core.utcdatetime import UTCDateTime
from tqdm import tqdm
from time import sleep

## rewirte_trace_name function
`rewirte_trace_name` Rewrite the trace name to the format of the STEAD csv file


In [7]:
def rewirte_trace_name(tr):
    """
    Rewrite the trace name to the format of the STEAD csv file
    :param tr: DataFrame Series
    :return: trace_name: str
    """
    # sample format for trace_name
    # USGS.109c.TA_20061101031222459_EV
    # 16 digits phasetime + last random digit avoid duplicate
    time = ''.join(re.findall(r'\d+', tr.phase_time))
    time = str(time[:16])
    time = time + str(np.random.randint(0, 9))
    trace_name = "USGS." + str(tr.station) + "." + str(tr.network) + "_" + time + "_EV"
    return trace_name

## rewirte_arrivals function
`rewirte_arrivals` return all find arrival time and convert them to position in the trace (int)

In [8]:
def rewirte_arrivals(index):
    """
    Rewrite the arrival time to the format of the STEAD csv file
    :param i: index
    :return: arrivals: Numpy array, e.g., p_pn_pg_s_sn_sg=[500, None, 550, 1000, None, None]

    """
    global usgs_df
    arr_dict = {"P": np.NaN, "Pn": np.NaN, "Pg": np.NaN, "S": np.NaN, "Sn": np.NaN, "Sg": np.NaN}
    phasetime = UTCDateTime(usgs_df.loc[index].phase_time)
    nscl_id = usgs_df.loc[index].nscl_code
    df_same_sta = usgs_df[usgs_df.nscl_code == nscl_id]
    # USGS dataset the original arrival is set at 60 sec which is 60 * 40 (sample_rate) = 2400
    # if arrival is at other position, change here
    arr = 2400
    arr_dict[usgs_df.loc[index].phase_hint] = arr
    # find the arrival time of other phases in same trace
    # in USGS catalog, other phases are list at different rows
    # the following code is to find the arrival time of other phases in same trace
    for inx, new_row in df_same_sta.iterrows():
        if inx == index:
            continue
        new_phase_time = UTCDateTime(usgs_df.loc[inx].phase_time)
        if abs(new_phase_time - phasetime) <= 60:
            new_arr = arr + int((new_phase_time - phasetime) * 40)
            arr_dict[usgs_df.loc[inx].phase_hint] = new_arr
            usgs_df = usgs_df.drop(inx)
    arrivals = list(arr_dict.values())
    # return all find arrival time and convert them to position in the trace (int)
    return arrivals

## rebuild_csv function
`rebuild_csv` generate a new csv file for metadata(information), a HDF5 file for training use.

In [9]:
def rebuild_csv(csv_path, csv_name='data.csv'):
    """
    Rebuild the csv file from the original csv file, i.e. USGS
    csv_path: the path of the original csv file
    csv_name: the name of the new csv file
    """
    # define hdf5 file name
    output_merge = csv_name[:-4] + '.hdf5'
    # create hdf5 file
    HDF0 = h5py.File(output_merge, 'a')
    HDF0.create_group("data")
    # read csv file
    global usgs_df
    usgs_df = pd.read_csv(csv_path)
    h5name = csv_path[:-4] + '.h5'
    dtfl = h5py.File(h5name, 'r')

    csvfile = open(csv_name, 'w')
    output_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # add an additional phase type (i.e. P vs Pn vs Pg etc)
    # label options
    # labels = ['network_code','receiver_code','receiver_type','receiver_latitude','receiver_longitude',
    #                              'receiver_elevation_m','phase_type','p_arrival_sample','p_status','p_weight','p_travel_sec',
    #                              's_arrival_sample','s_status','s_weight',
    #                              'source_id','source_origin_time','source_origin_uncertainty_sec',
    #                              'source_latitude','source_longitude','source_error_sec',
    #                              'source_gap_deg','source_horizontal_uncertainty_km', 'source_depth_km', 'source_depth_uncertainty_km',
    #                              'source_magnitude', 'source_magnitude_type', 'source_magnitude_author','source_mechanism_strike_dip_rake',
    #                              'source_distance_deg', 'source_distance_km', 'back_azimuth_deg', 'snr_db', 'coda_end_sample',
    #                              'trace_start_time', 'trace_category', 'trace_name']
    # Adaptive label options
    # p_pn_pg_s_sn_sg : list, arrival sample points of P, Pn, Pg, S, Sn, Sg phase
    # [500, None, 550, 1000, None, None]  value is None if no arrival, value should be integer
    picklabels = ['p_pn_pg_s_sn_sg', 'snr_db', 'source_magnitude',
                  'receiver_type', 'source_distance_deg', 'source_distance_km',
                  'trace_category', 'trace_name', 'source_id'
                  ]
    output_writer.writerow(picklabels)
    csvfile.flush()
    print('output files have been generated!')
    # label of trace_category in the csv file
    trace_category = 'earthquake'  # earthquake label
    # main loop for each file (trace)
    with tqdm(total=len(usgs_df)) as pbar:
        for i in range(len(usgs_df)):
            pbar.update(1)
            if i not in usgs_df.index:
                continue
            try:
                trace_name = rewirte_trace_name(usgs_df.loc[i])
                p_pn_pg_s_sn_sg = rewirte_arrivals(i)
            except Exception as e:
                print(e)
                print(usgs_df.loc[i])
                print(i)

            output_writer.writerow([p_pn_pg_s_sn_sg,
                                    usgs_df.loc[i].snr_db,
                                    usgs_df.loc[i].source_magnitude,
                                    usgs_df.loc[i].channel[:2],  # receiver_type
                                    usgs_df.loc[i].source_distance_deg,  # float64
                                    usgs_df.loc[i].source_distance_km,  # float64
                                    trace_category,  # trace_category
                                    trace_name,  # trace_name
                                    usgs_df.loc[i].phase_id,  # source_id
                                    ])
            # save the trace data to hdf5 file
            x = dtfl.get(usgs_df.loc[i].event_id + '/' + usgs_df.loc[i].waves_id + '/' + usgs_df.loc[i].phase_id)
            data = np.array(x)
            # write metadata (attributes)
            HDFr = h5py.File(output_merge, 'a')
            dsF = HDFr.create_dataset("data/" + trace_name, data.shape, data=data, dtype=np.float32)
            dsF.attrs['p_pn_pg_s_sn_sg'] = np.array(p_pn_pg_s_sn_sg)
            dsF.attrs['snr_db'] = usgs_df.loc[i].snr_db
            dsF.attrs['source_magnitude'] = usgs_df.loc[i].source_magnitude
            dsF.attrs['receiver_type'] = usgs_df.loc[i].channel[:2]
            dsF.attrs['source_distance_deg'] = usgs_df.loc[i].source_distance_deg
            dsF.attrs['source_distance_km'] = usgs_df.loc[i].source_distance_km
            dsF.attrs['trace_category'] = trace_category
            dsF.attrs['trace_name'] = trace_name
            dsF.attrs['source_id'] = usgs_df.loc[i].phase_id
            HDFr.flush()
    HDFr.close()
    csvfile.close()

In [10]:
def main(file_path='/Users/hao/Downloads/2013/'):
    """
    Convert dataset into HDF5
    file_path: directory of csv files 
    """
    time_start = time.time()
    file_path = file_path + '*.csv'
    # finished = ['/Users/hao/Downloads/2013/201309_mlaapde.csv']
    # finished: complete csv file
    for csv_file in glob.glob(file_path):
        if csv_file in finished:
            continue
        print('working on ' + csv_file + '...')
        title = ''.join(re.findall(r'\d+', csv_file)[0])
        # write uniform new csv name
        title = 'usgs'+ title + '.csv'
        rebuild_csv(csv_file, title)
    print('Done!')
    time_end = time.time()
    print('Total time cost: ', (time_end - time_start)/60, 'minutes')
    return


In [ ]:
if __name__ == '__main__':
    # input csv file path and put data files in the same directory
    main('/Volumes/TOSHIBA EXT/STEAD/USGS/')